## BRONZE TO SILVER LAYER


In [4]:
import os
import pathlib
import numpy as np
import pandas as pd

In [5]:
## Folder and File path
cwd = pathlib.Path(os.getcwd())
gold_path = cwd.joinpath("../DATA/GOLD")
gold_path.mkdir(parents=True, exist_ok=True)

### Silver Layer - INVESTMENT and HOLDING History


In [6]:
# read the csv file
gold_data_trade_history_file_path = gold_path.joinpath("TradeHistory.csv")
df_gold_tread_history = pd.read_csv(gold_data_trade_history_file_path)

# convert Datetime to Date string
df_gold_tread_history["date"] = pd.to_datetime(
    df_gold_tread_history["datetime"]
).dt.date

golden_table_names = {"Investment": "avg_price", "Holdings": "holding_quantity"}

date_range = pd.date_range(
    start=df_gold_tread_history["date"].min(), end=pd.to_datetime("today"), freq="D"
)

for file_name, table_name in golden_table_names.items():
    # Create a new DataFrame with an updated date range
    df = pd.DataFrame({"date": date_range.date})
    grouped = df_gold_tread_history.groupby("stock_name")
    for stock_name, group in grouped:
        df = pd.merge(df, group[["date", table_name]], on="date", how="left").rename(
            columns={table_name: stock_name},
        )

    # # setting date column as index
    df.set_index("date", inplace=True)
    # Reindexing to fill the missing data with the last available data
    df = df.ffill()
    df.replace(0.0, np.nan, inplace=True)

    df["date"] = df.index.astype("str")

    df.to_csv(gold_path.joinpath(f"{file_name}.csv"), orient="records", indent=4)